In [17]:
import pandas as pd
import numpy as np
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
training_val_df = pd.read_csv('./data/Training_set_values.csv', index_col='id')
training_label_df = pd.read_csv('./data/Training_set_labels.csv', index_col='id')
training_df = training_val_df.join(training_label_df, how='inner')

height has been deprecated.



In [18]:
# data cleaning

# remove columns with identical column
# region = region_cd
rm_cols = ['region']
for col in rm_cols:
    training_df.drop(col, axis=1, inplace=True)

# remove columns with no uniqueness
training_cols = list(training_df.columns.values)
for col in training_cols:
    unique_elements = len(training_df[col].unique())
    if unique_elements == 1:
        print('Removed column with constant value: ', col)
        training_df.drop(col, axis=1, inplace=True)

# deal with null values
null_values = [0, 'none' 'None', 'na', 'NA', 'Na', 'Unknown', 'Not Known', '', ' ', 'unknown']
for null_val in null_values:
    training_df.replace(null_val, np.nan)
        
# drop columns that are completely NA
training_df = training_df.dropna(axis=1, how='all')

Removed column with constant value:  recorded_by


In [20]:
# clean up free form columns
ff_cols = ['funder', 'installer', 'wpt_name', 'scheme_name']
free_form_df = training_df[ff_cols]
for col in ff_cols:
    training_df.drop(col, axis=1, inplace=True)

In [21]:
# convert remaining non-numeric cols to categorical
training_cols = list(training_df.columns.values)
for col in training_cols:
    if training_df[col].dtype == np.object:
        training_df[col].astype('category')

In [24]:
# convert to numerical vector of 1,0 for all categorical values
vect_training_df = pd.get_dummies(training_df, sparse=True)
print(vect_training_df)

KeyboardInterrupt: 